In [ ]:
pip install googletrans==4.0.0-rc1

### 필요사항
- 번역 정확도 높이기
- 파파고 번역기 api 사용후 글 확인 (일본, 프랑스)
- 전처리 과정

In [18]:
import nltk
#nltk.download("book", quiet=True)
#from nltk.book import *
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.tag import pos_tag
# from textblob import TextBlob
from urllib.parse import quote_plus
from bs4 import BeautifulSoup
from selenium import webdriver
import googletrans

def Url(search):
  #  url_temp = 'https://www.google.com/search?q=%7Bsearch%7D&tbm=nws%27.format(search=search)
#     url_temp = 'https://search.mt.co.kr/searchNewsList.html?srchFd=TOTAL&range=IN&reSrchFlag=&preKwd=&search_type=m&kwd=%7Bsearch%7D&bgndt=20190401&enddt=20190930&category=MTNW&sortType=allwordsyn&subYear=&category=MTNW&subType=mt%27.format(search=search)
    for pageNo in range(1,10,10):
        url = url_temp + "&start={pageNo}".format(pageNo=pageNo)
        driver = webdriver.Chrome()
        driver.get(url)

        html = driver.page_source
        soup = BeautifulSoup(html)

        news_urls = []
        # 검색 했을 때 제목과 breadcrumbs, 접속주소를 가져올 것이다.
        # #search>div>div>div>div>div
        list = soup.select('#search .MjjYud>div>div>div')
        if not list:
            print("없어어어어어어엉~")
            break

        for i in list:
            if i.select_one('div>a>div>div:nth-child(2)>div:nth-child(2)'):
                news_urls.append(i.select_one(".WlydOe").get("href"))
#                 print(i.select_one('div>a>div>div:nth-child(2)>div:nth-child(2)').attrs)
#                 print(i.select_one('.iRPxbe>div:nth-child(2)').text)
            else:
                print("text 속성 없어어어어어어엉~")
        print(news_urls)
        if not news_urls:
            break

In [19]:
def news(news_urls):
    translator = googletrans.Translator() # 번역하는 함수
    for url_no in news_urls:
        print(url_no)
        driver = webdriver.Chrome()
        driver.get(url_no)
        html = driver.page_source
        soup = BeautifulSoup(html)

        str = "" # p 태그의 내용을 담는 변수
        result = "" # 영어로 변환하여 저장할 변수
        for text in soup.findAll("p"): # soup의 모든 p 태그를 처리
            str += text.get_text() + "\n" 
            if(len(str)>1000): # 번역을 위해서 str의 글자수가 1000자보다 넘을 경우에만 번역 실행
                # 영어로 변환, 정확도를 위해서 어떤 언어인지 알려주는 것도 좋음
                # ref: https://coding-kindergarten.tistory.com/98
#                 result1 = translator.translate(str, dest='ko')
                print(str)
                result += translate(str)
#                 result += result1.text.replace(" -", "-").replace("- ", "-")
                str = ""
        
        # 마지막으로 남아있는 문장 번역
        if str !="":
            result += translate(str)
#             result1 = translator.translate(str, dest='ko')
#             result += result1.text.replace(" -", "-").replace("- ", "-")
        return result

In [25]:
import json
import os
import sys
import urllib.request
client_id = "6yrWrXVyj5VaRKnqi_fv"
client_secret = "i9h16BCiqz"

url = "https://openapi.naver.com/v1/papago/n2mt"
request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)

def translate(str):
    encText = urllib.parse.quote(str)
    data = "source=ja&target=en&text=" + encText
    response = urllib.request.urlopen(request, data=data.encode("utf-8"))
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
        result = json.loads(response_body.decode('utf-8'))
        return result['message']['result']['translatedText']
#         print(result['message']['result']['translatedText'])
    #     print(response_body.decode('utf-8'))
    else:
        print("Error Code:" + rescode)

In [156]:
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import time

def news(news_urls):
    translator = googletrans.Translator() # 번역하는 함수
    for url_no in news_urls:
        print(url_no)
        options = webdriver.ChromeOptions()  
#         options.add_argument('headless')              # Headless 
#         options.add_argument("--window-size=800,600") # window-size -> 기본 : 1920,1080
#         options.add_argument("disable-gpu")           # browser gpu stop
        # 실제 브라우저처럼 head 정보 입력(크롤링 차단 방지)
#         options.add_argument("User-Agent:  Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) \
#                              AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.131 Safari/537.36")
        options.add_argument("lang=ko_KR")      
#         options.add_argument("disable-gpu")
        # 혹은 options.add_argument("--disable-gpu")

        
        driver = webdriver.Chrome('chromedriver', chrome_options=options)
        action = webdriver.ActionChains(driver)
        action.send_keys(Keys.SHIFT, Keys.F10, 't').perform()
        driver.get(url_no)
        pyautogui.hotkey('shift', 'F10')
        for i in range(8):
            pyautogui.hotkey('down')
        pyautogui.hotkey('enter')
        
        time.sleep(5)
        html = driver.page_source
        soup = BeautifulSoup(html)

        str = "" # p 태그의 내용을 담는 변수
        for text in soup.findAll("p"): # soup의 모든 p 태그를 처리
            str += text.get_text()
        return str

In [157]:
text = news(['https://www.kdramastars.com/articles/126325/20220820/k-dramas-north-south-korea-money-heist-korea-crash-landing-on-you.htm'])
text

https://www.kdramastars.com/articles/126325/20220820/k-dramas-north-south-korea-money-heist-korea-crash-landing-on-you.htm


C:\Users\multicampus\AppData\Local\Temp\ipykernel_40644\2652359399.py:22: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome('chromedriver', chrome_options=options)


'\n        To enjoy our website, you\'ll need to enable JavaScript in your web browser. Please click here to learn how.\n    K-dramas managed to cover the stories of North Korea despite the ongoing war between the two countries.\xa0Interestingly, the remarkable and iconic series in Hallyu shows a glimpse of life in North Korea.\xa0From rom-com to crime thriller series, these 5 K-dramas feature stories about North and South Korea.\xa0Netflix introduced the Korean remake of the hit Spanish series "La Casa de Papel" or "Money Heist" created by Álex Pina.\xa0After five successful seasons, the tv show was remade to "Money Heist: Korea - Joint Economic Area" starring Yoo Ji Tae as the Professor, "Squid Game" star Park Hae Soo as Berlin, Jeon Jong Seo as Tokyo, and many more.\xa0The Korean remake follows a hostage crisis set in a unified Korean Peninsula.From hostages, employees, gang members, and even the task force are mixed from both North and South.\xa0While the first part, containing 6 e

In [160]:
print(text)


        To enjoy our website, you'll need to enable JavaScript in your web browser. Please click here to learn how.
    K-dramas managed to cover the stories of North Korea despite the ongoing war between the two countries. Interestingly, the remarkable and iconic series in Hallyu shows a glimpse of life in North Korea. From rom-com to crime thriller series, these 5 K-dramas feature stories about North and South Korea. Netflix introduced the Korean remake of the hit Spanish series "La Casa de Papel" or "Money Heist" created by Álex Pina. After five successful seasons, the tv show was remade to "Money Heist: Korea - Joint Economic Area" starring Yoo Ji Tae as the Professor, "Squid Game" star Park Hae Soo as Berlin, Jeon Jong Seo as Tokyo, and many more. The Korean remake follows a hostage crisis set in a unified Korean Peninsula.From hostages, employees, gang members, and even the task force are mixed from both North and South. While the first part, containing 6 episodes, was released 

In [158]:
url_no = 'https://www.kdramastars.com/articles/126325/20220820/k-dramas-north-south-korea-money-heist-korea-crash-landing-on-you.htm'
driver = webdriver.Chrome()
driver.get(url_no)
# pyautogui.hotkey('shift', 'F10')
# for i in range(8):
#     pyautogui.hotkey('down')
# pyautogui.hotkey('enter')

In [159]:
driver.get('https://trends.tribunnews.com/2022/09/06/sinopsis-crash-landing-on-you-kisah-wanita-nyasar-ke-korea-utara-dibintangi-son-ye-jin-hyun-bin')

In [107]:
import spacy
import re

nlp = spacy.load('en_core_web_md')
# text = 'Yuh-jung Youn won the Oscar for best supporting actress for her performance in "Minari" on Sunday and made history by becoming the first Korean actor to win an Academy Award.'
# text = 'Korean drama "U-Yongu Lawyer is a genius skin", which depicts a new lawyer with autism spectrums, growing, and has a great response in Japan.'
# text = 'In response to the statement of Lee Sanbek, the drama official commented on the Korean media carefully commented on the Korean media. However, "The scriptwriter Moon Ji-won and Yu Insok are positive attitude", and the possibilities of Season 2 are certainly talked about among the production teams!'
# text = 'If watching Squid Game means the prospect of playing red light, green light now fills you with nerve-shredding terror rather than fond childhood memories, you aren\t alone.The Korean thriller, which tells the story of debt-ridden people competing for a huge cash prize in a deadly series of children\s games, has become Netflix\s biggest ever series launch - streamed by 111 million users in its first 28 days.In doing so it knocked Bridgerton off the top spot, making clear that Korean dramas have most certainly been given the green light by audiences worldwide. So, how can we understand this rise, and what are some of the other K-dramas to look out for if you\'re a new convert?'

# text = re.sub('"[^a-zA-Z]"', ' ', text)
text = text.replace('\"', "")
text = text.lower()
doc = nlp(text)

noun_chunks = doc.noun_chunks

print("=="*40)
str_format = "{:>30}{:>15}{:>15}{:>20}"
print(str_format.format('Text', 'Root Text', 'Root Dep', 'Root Head Text'))
print("=="*40)

data = []
for chunk in doc.noun_chunks:
    data.append(chunk.text)
    print(str_format.format(chunk.text, chunk.root.text, chunk.root.dep_, chunk.root.head.text))

                          Text      Root Text       Root Dep      Root Head Text
                   our website        website           dobj               enjoy
                           you            you          nsubj                need
                    javascript     javascript           dobj              enable
              your web browser        browser           pobj                  in
            k-드라마는 남북 전쟁이 계속되고           계속되고           pobj               
    
                 불구하고 북한의 이야기를           이야기를          appos                있음에도
                         있었습니다          있었습니다           pobj                  다룰
             흥미롭게도 놀랍고 상징적인 한류             한류           ROOT                  한류
                          시리즈는           시리즈는          appos                  한류
                     북한의 삶을 엿볼             엿볼          appos                  한류
                          있습니다           있습니다          appos                  한류
                     스릴러 시리즈

In [54]:
text

['reproduction', 'n']

In [108]:
from nltk.corpus import stopwords

data
result = []
for token in data:
    result.append(token.split())

# 불용어 제거
stops = set(stopwords.words('english'))
for data in result:
    for word in data:
        if word in stops:
            data.remove(word)

In [109]:
import nltk

tag=[nltk.pos_tag(data) for data in result if data]
print(tag)

[[('website', 'NN')], [('javascript', 'NN')], [('web', 'NNS'), ('browser', 'NN')], [('k-드라마는', 'JJ'), ('남북', 'NNP'), ('전쟁이', 'NNP'), ('계속되고', 'NN')], [('불구하고', 'JJ'), ('북한의', 'NNP'), ('이야기를', 'NN')], [('있었습니다', 'NN')], [('흥미롭게도', 'JJ'), ('놀랍고', 'NNP'), ('상징적인', 'NNP'), ('한류', 'NN')], [('시리즈는', 'NN')], [('북한의', 'JJ'), ('삶을', 'NNP'), ('엿볼', 'NN')], [('있습니다', 'NN')], [('스릴러', 'NN'), ('시리즈까지', 'NN')], [('5개의', 'CD'), ('k-드라마는', 'NN')], [('대한', 'NN')], [('이야기를', 'NN'), ('다룹니다', 'NN')], [('넷플릭스(netflix)는', 'NN')], [('피나(alex', 'JJ'), ('pina)가', 'NN')], [('히트작', 'NN')], [("'라", 'POS'), ('카사', 'NN'), ('드', 'NNP'), ('파펠(la', 'NNP'), ('casa', 'NN'), ('de', 'FW'), ('papel', 'NN')], [('또는', 'JJ'), ("'머니", 'POS'), ('습격(money', 'NN'), ("heist)'의", 'NN')], [('리메이크', 'NN')], [('작품을', 'NN'), ('소개했다', 'NN')], [('5개의', 'CD')], [('시즌', 'JJ'), ('후에', 'NNP'), ('tv', 'NN')], [('유지태', 'NN')], [('교수,', 'JJ'), ('오징어', 'NNP'), ('게임의', 'NNP'), ('박해수(베를린', 'NN')], [('습격', 'NN')], [('지역으로', 'NN')], [('리메이크되었습니다', '

- pos: n(명사), v(동사), a(형용사), r(부사), satellite adjectives(s)
- NN, NNS, NNP, NNPS (n)
- VB, VBD, VBG, VBN, VBP, VBZ (v)
- RB, RBR, RBS (r)
- JJ (a)

In [48]:
n = {'NN', 'NNS', 'NNP', 'NNPS'}
v = {'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'}
r = {'RB', 'RBR', 'RBS'}
a = {'JJ'}

In [49]:
list(tag)

[[('perpetrator', 'NN')],
 [('movie', 'NN')],
 [('utamaru', 'NN')],
 [('problem', 'NN')],
 [('movie', 'NN')],
 [('gifts', 'NNS')],
 [('important', 'JJ'), ('people', 'NNS')],
 [('disaster', 'NN'),
  ('prevention', 'NN'),
  ('[emergency', 'NN'),
  ('support', 'NN')],
 [('pakistan', 'RB'), ('flood', 'NN'), ('damage', 'NN')],
 [('right', 'RB'), ('knowledge', 'NN')],
 [('preparation', 'NN')],
 [('evacuation', 'NN'), ('drills', 'NNS')],
 [('typhoons', 'NNS')],
 [('torrential', 'JJ'), ('rains', 'NNS')],
 [('3', 'CD'), ('minutes', 'NNS'), ('keyword', 'VBD')],
 [('many', 'JJ'), ('coupons', 'NNS')],
 [('ids', 'NNS')],
 [('javascript', 'NN')],
 [('the', 'DT'), ('functions', 'NNS')],
 [('yahoo', 'NN')],
 [('news', 'NN')],
 [('javascript', 'NN'), ('settings', 'NNS')],
 [('javascript', 'NN'), ('settings', 'NNS')],
 [('8/19', 'CD'), ('(fri)', 'JJ'), ('19:20', 'CD'), ('distribution', 'NN')],
 [('new', 'JJ'), ('lawyer', 'NN')],
 [('autism', 'NN'), ('spectrums', 'NNS')],
 [('huge', 'JJ'), ('response', '

In [99]:
test = []
for tags in tag:
    result = []
    for word in tags:
        word = list(word)
        if word[1] in n:
            word[1] = 'n'
        elif word[1] in v:
            word[1] = 'v'
        elif word[1] in r:
            word[1] = 'r'
        elif word[1] in a:
            word[1] = 'a'
        else:
            word[1] = 'n'
        result.append(word)
    test.append(result)

In [51]:
test

[[['perpetrator', 'n']],
 [['movie', 'n']],
 [['utamaru', 'n']],
 [['problem', 'n']],
 [['movie', 'n']],
 [['gifts', 'n']],
 [['important', 'a'], ['people', 'n']],
 [['disaster', 'n'],
  ['prevention', 'n'],
  ['[emergency', 'n'],
  ['support', 'n']],
 [['pakistan', 'r'], ['flood', 'n'], ['damage', 'n']],
 [['right', 'r'], ['knowledge', 'n']],
 [['preparation', 'n']],
 [['evacuation', 'n'], ['drills', 'n']],
 [['typhoons', 'n']],
 [['torrential', 'a'], ['rains', 'n']],
 [['3', 'n'], ['minutes', 'n'], ['keyword', 'v']],
 [['many', 'a'], ['coupons', 'n']],
 [['ids', 'n']],
 [['javascript', 'n']],
 [['the', 'n'], ['functions', 'n']],
 [['yahoo', 'n']],
 [['news', 'n']],
 [['javascript', 'n'], ['settings', 'n']],
 [['javascript', 'n'], ['settings', 'n']],
 [['8/19', 'n'], ['(fri)', 'a'], ['19:20', 'n'], ['distribution', 'n']],
 [['new', 'a'], ['lawyer', 'n']],
 [['autism', 'n'], ['spectrums', 'n']],
 [['huge', 'a'], ['response', 'n']],
 [['japan', 'n']],
 [['great', 'a'], ['response', 'n']

In [100]:
from nltk.stem import WordNetLemmatizer

lemma = WordNetLemmatizer()
result = []
for texts in test:
    keyword = ""
    for text in texts:
        keyword += lemma.lemmatize(text[0],text[1]) + " "
#         print(lemma.lemmatize(text[0],text[1]))
    result.append(keyword.strip())

In [90]:
result

['synopsis',
 'rich woman',
 'south korea',
 'south korea',
 'dda',
 'son ye jin',
 'hyun bin',
 'efficient quota',
 'streamingprofil korean drama drama crash',
 'broadcast schedule',
 'link',
 'drakor',
 'interesting information',
 'lover',
 'romantic comedy series',
 'series',
 'hyun bin',
 'son ye jin',
 'husband',
 'wife',
 'profile',
 'korean drama player',
 'spotlight',
 'prime minister',
 'prime minister',
 'love',
 'reporter',
 'yoona snsdnamun',
 'synopsis',
 'drama',
 'mydramalist',
 'broadcast schedule',
 'drakor.crash landing',
 'figure',
 'yoon se ri',
 'woman',
 'south korea',
 'many business',
 'young age.one day',
 'accident',
 'north korean area',
 'tree',
 'ri jeong hyeok',
 'hyun bin',
 'soldier',
 'son',
 'north korean official',
 'jeong hyeok',
 'yoon se ri',
 'even jeong hyeok',
 'men',
 'jeong hyeok',
 'arrange marriage',
 'north korean conglomerate daughter',
 'seo',
 '(seo ji hye',
 "se ri's candidate",
 'namely gu seung jung',
 'kim jung hyun',
 'north korea',

In [91]:
import json
import os
import sys
import urllib.request
client_id = "6yrWrXVyj5VaRKnqi_fv"
client_secret = "i9h16BCiqz"

url = "https://openapi.naver.com/v1/papago/n2mt"
request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)

def translate(str):
    encText = urllib.parse.quote(str)
    data = "source=en&target=ko&text=" + encText
    response = urllib.request.urlopen(request, data=data.encode("utf-8"))
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
        result = json.loads(response_body.decode('utf-8'))
        return result['message']['result']['translatedText']
#         print(result['message']['result']['translatedText'])
    #     print(response_body.decode('utf-8'))
    else:
        print("Error Code:" + rescode)

In [103]:
translate("squid game. prospect. red light. green light. nerve-shredding terror. fond childhood memory. alone.the korean thriller. story. debt-ridden people. huge cash prize. deadly series. children's game. 111 million user. bridgerton. top spot. korean drama. green light. audience. rise. other k-dramas. new convert")

HTTPError: HTTP Error 429: Too Many Requests

In [38]:
str = "u-yongu \n squid game. prospect. red light. green light. nerve-shredding terror. fond childhood memory. alone.the korean thriller. story. debt-ridden people. huge cash prize. deadly series. children's game. 111 million user. bridgerton. top spot. korean drama. green light. audience. rise. other k-dramas. new convert"

In [39]:
translator = googletrans.Translator() # 번역하는 함수
translator.translate(str, src='en', dest='ko').text

'U-Yongu.오징어 게임.전망.빨간불.초록불.신경 분쇄 테러.어린 시절 기억을 좋아합니다.한국 스릴러.이야기.빚을지고있는 사람들.거대한 상금.치명적인 시리즈.어린이 게임.1 억 1,100 만 명의 사용자.브리거 턴.최고점.한국 드라마.초록불.청중.증가.다른 K- 드라마.새로운 변환'

In [101]:
txt = "\n".join(result)

print(translator.translate(txt, src='en', dest='ko').text)

웹 사이트
자바 스크립트
웹 브라우저
K- 드라마
이야기
북한
진행중인 전쟁
두 나라
놀라운 상징적 인 시리즈
hallyu
일견
삶
북한
롬콤
스릴러 시리즈
5 K-Dramas 기능 스토리
북쪽
대한민국
넷플릭스
한국 리메이크
스페인 시리즈를 쳤다
라 카사 데 파펠
돈 습격
Álex Pina
5 번의 성공적인 시즌
티비 쇼
돈 습격
한국 - 공동 경제 지역
유 지 테
교수
베를린
전종 서
도쿄
한국 리메이크
인질 위기
인질로부터 반도
직원
갱 멤버
조차 태스크 포스
북쪽
남쪽
첫 번째 부분
6 에피소드
6월
돈 습격
한국 - 공동 경제 지역
마지막 분기
강의
개요
K- 드라마
북한의 이야기
불시착
아들 Ye Jin
현 빈
대부분의 과장된 시리즈
드라마
이야기
한국 상속인 윤 세리
아들 Ye Jin
DMZ
한반도
북한 군인
Ri Jung Hyuk 선장
고향
도전
긴장
두 나라
윤 세리
캡틴 리
이승
지
Jo Jung Suk
2012 롬콤 k
드라마
왕
2 하트
협동
영향력있는 두 사람
북한
대한민국
Ha ha ha woon
역할
북한의 엘리트 특수 부대 강사
딸
고위 군사 공무원
공동 군사 협력
한국의 왕자
이재
사랑 증오 관계
듀오
힘든 상황
사랑
충의
또 다른 2012 K- 드라마
이야기
두 카운티
한국 과학자 Seo Myung Joon
사랑
북한 과학자
진 전
김정은
이종 수크
파크 헌
천재 심장 외과 의사
북한
어린이
아버지
북한
의사
새로운 나라
사건
대한민국
최상위 병원
대학 병원
Kdrama 스타 이메일 업데이트


In [102]:
print(translate(txt))

웹사이트
자바스크립트
웹 브라우저
k-back의
이야기
북한
진행 중인 전쟁
두 나라
주목할 만한 상징적인 시리즈
한류
잠깐보다
인생
북한
롬콤의
스릴러 시리즈
5k급 특집 기사
북쪽
대한민국.
넷플릭스
한국판 리메이크
스페인 시리즈를 히트시키다
라카사드 종이
돈을 강탈하다
알레스피나
다섯 번의 성공적인 시즌
텔레비전 쇼
돈을 강탈하다
한국과 공동 경제 지역
유지태
교수님
베를린의
전종서
東京의
한국판 리메이크
인질극
인질로부터 반도.
직원
갱단원
짝수 태스크포스
북쪽
남쪽
제1부
6화
6월.
돈을 강탈하다
한국과 공동 경제 지역
최종 분기
코스
고갈시키다
k-back의
북남 이야기
불시착
손예진
현빈
가장 과장된 급수
연극하다
이야기
한국의 상속녀 윤세리
손예진
dmz
한반도
북한군
대장 리정혁
고향
도전
긴장
두 나라
윤세리
대위리
이승
지
조정숙
2012년 롬콤
연극하다
왕
하트 2개
공동 작업
영향력 있는 두 사람
북한
대한민국.
하지운
역할.
북한의 정예 특수부대 교관
딸.
군 고위 관리
합동 군사 협력
한국의 왕세자
이재
애증 관계
2인조
어려운 상황
사랑하다
충성심
또 다른 2012년 한국 드라마
이야기
두 군
한국의 과학자 서명준
사랑하다
북한 과학자
진재
김정은
이종석
박훈
천재 심장외과 의사
북한
어린아이
아버지.
북한
의사선생님.
신흥국
사건
대한민국.
일류 병원
대학 병원
kdrama star 이메일 업데이트


가해자
영화
우타 마루
문제
영화
선물
중요한 사람들
재난 예방 [비상 지원
파키스탄 홍수 피해
올바른 지식
준비
대피 훈련
태풍
폭우
3 분 키워드
많은 쿠폰
ID
자바 스크립트
함수
야후
소식
자바 스크립트 설정
자바 스크립트 설정
8/19 (Fri) 19:20 분포
새로운 변호사
자폐 스펙트럼
큰 반응
일본
훌륭한 응답
일본
드라마
유
-
yongu
천재 피부
[사진
인기있는 한국 여배우
이것
일하다
최대 잠재 고객 등급
17.5 %
팔월
가능성
계절
스포일러
기사
보고서
한국 매체
팔월
이 산 벡
대표
이야기
제조업 회사
U-Yong-woo
천재 피부
계절
가능성
계속
일정
깁스
생산 팀
논의
목표
90 %
승무원
큰 변화
주목
인스 타 그램
/@booboo2injo Lee Sanbek
드라마 공무원
한국 매체
대본 작자
달
긍정적 인 태도
가능성
계절
생산 팀
캐스트 멤버
90 % 이상
시각
구체적인 토론
강 테오
역할
이 주노
큰 휴식
일하다
사무실
남성
창조
콘크리트
콘텐츠
콘텐츠
Instagram/@channel.ena.d
강 지연
역할
Bos Jung Myeong-Suk
U-young woo
회견
한국 매체
쾌히 하기
. 인스 타 그램
계절
대기
드라마 촬영 사이트
느끼다
강의
끝
모습
Ju Jong Hyuk
좋은 동료
Kwon Minu
경쟁
라디오
19 일
가능성
계절
_
최종 스크립트
계절
일하다
웹 만화
뮤지컬
계속
깁스
계절
기사
'에스
반응
유
-성우
천재
피부 강 기금 반응
일하다
Kstyle9/1
thu
강 테오
구월
입대
팬 미팅
U-Yong-woo
천재 피부
군대 Kstyle9/1
(Thu) 9:36 Kstyle9/5
한국 도라 우용우
천재
스킨 글로벌 1 위
Account8/31
20:13 통통한
단신
겨드랑이 털
패션 잡지
모델
다양성 시대
재분
다양성 에리카 토다
여배우 부재
내년 1 월 도라 반환 이론
특정 상황
진짜 페이스 샷
금 폭탄 / 배럴 술 롤빵
유리
파마
매우 잘 생긴 아름다운 선
<메 구미 마츠모토
시각
결혼